In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import graphblas as gb
from graphblas import Matrix, Vector, Scalar
from graphblas import unary, binary, monoid, semiring

## Create and visualize a Matrix

In [ ]:
data = [
    [3, 0, 3, 5, 6, 0, 6, 1, 6, 2, 4, 1],
    [0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5, 6],
    [3, 2, 3, 1, 5, 3, 7, 8, 3, 1, 7, 4],
]

In [ ]:
rows, cols, weights = data
m = Matrix.from_coo(rows, cols, weights)
m
# Size of the sparse matrix is 7x7 with 12 non-zero elements of type INT64

In [ ]:
m
# This is an adjacency matrix
# Reading along a row shows the out-nodes of a vertex
# Reading along a column shows the in-nodes of a vertex

In [ ]:
gb.viz.draw(m)

## Create and visualize a Vector

In [ ]:
v = Vector.from_coo([1], [0], dtype=m.dtype, size=m.nrows)

In [ ]:
v

## Single-source Shortest Path

This uses the **_min-plus_** semiring because we want to add the edges, then take the minimum length of available paths.

In [ ]:
# Create a vector and initialize a starting vertex (1) with a distance of zero
v = Vector(m.dtype, m.nrows)
v[1] << 0
v

In [ ]:
m

In [ ]:
# v @ m will give us one step in a Breadth-first search
w = v.dup()
w << v.vxm(m, semiring.min_plus)
w

In [ ]:
# Look again at m and see that vertex 1 points to vertices 4 and 6 with the weights indicated
gb.viz.draw(m)

We have the right semiring, but we already lost the initial distance=0 for vertex 1. How do we keep that information around as we step thru the BFS?

GraphBLAS has a builtin accumulator available for every operation.
Because it's C-based, you pass in the output object and it accumulates its existing values with the result, then returns itself.

In [ ]:
w = v.dup()
w(binary.min) << v.vxm(m, semiring.min_plus)
w
# Now we see that the zero distance for vertex 1 is preserved

Let's take another step

In [ ]:
w(binary.min) << w.vxm(m, semiring.min_plus)
w
# We see that the path to vertex 4 is now shorter. That's `min` doing its thing.
# Verify the other path distances from vertex 1 with at most two hops

In [ ]:
gb.viz.draw(m)

The algorithm repeats until a new computation is the same as the previous result

In [ ]:
w = v.dup()
while True:
    w_old = w.dup()
    w(binary.min) << w.vxm(m, semiring.min_plus)
    if w.isequal(w_old):
        break
w

## Alternate solution without using an accumulator

In the min_plus semiring, the "empty" value of a sparse matrix is not actually 0, but +infinity.

That way, `min(anything, +inf) = anything`, similar to the normal addition 0 of `add(anything, 0) = anything`.

A clever trick sets the diagonal of the matrix to all zeros. This makes it behave like the Identity matrix for the min_plus semiring.

Observe:

In [ ]:
m_ident = Matrix.from_coo(range(7), range(7), [0] * 7)
m_ident

In [ ]:
v.clear()
v.build([1], [0])
v

In [ ]:
v << v.vxm(m_ident, semiring.min_plus)
v
# See how it preserved v exactly

In [ ]:
# Let's try again
v.clear()
v.build([0, 1, 4], [14, 0, 77])
v << v.vxm(m_ident, semiring.min_plus)
v

So zeros along the diagonal preserve what you already have in `v` without adding any new path information. That's the behavior we want, so let's update `m` with zeros on the diagonal and repeat SSSP without using accumulators.

In [ ]:
for i in range(m.nrows):
    m[i, i] << 0
m

In [ ]:
# Reset v
v.clear()
v[1] << 0
v

In [ ]:
# Take one step (notice no accumulator is specified)
v << v.vxm(m, semiring.min_plus)
v

In [ ]:
# Repeat until we're converged
while True:
    w = v.dup()
    v << v.vxm(m, semiring.min_plus)
    if v.isequal(w):
        break
v

### And that's SSSP in 5 very readable lines of Python, thanks to GraphBLAS